In [397]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score,  mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

## Load data

In [398]:
for file_name in os.listdir("files"):
    if file_name.endswith('.csv'):
        file_path = os.path.join("files", file_name)

        df_name = os.path.splitext(file_name)[0]
        globals()[df_name] = pd.read_csv(file_path)


## Simplification of categories

In [399]:
# Merge sports data and extract sports categories
def extract_sports_category(dataset, sports_code):
    merged = pd.merge(dataset, sports_code, left_on="Sports", right_on="Code")
    merged["Sports_Category"] = merged["Categorie"]
    return merged[["PRIMARY_KEY", "Sports_Category"]]

learn_sports = extract_sports_category(learn_dataset_sport, code_Sports)
test_sports = extract_sports_category(test_dataset_sport, code_Sports)

# Merge departments into regions and extract relevant region columns
def merge_and_extract_region(df, merge_column, region_column_name):
    merged = pd.merge(df, departments, left_on=merge_column, right_on="DEP")
    merged[region_column_name] = merged["REG"]
    return merged.drop(["Nom du département", "REG", "DEP", merge_column], axis=1)

learn_dataset_job = merge_and_extract_region(learn_dataset_job, "JOB_DEP", "REG_JOB")
learn_dataset_retired_jobs = merge_and_extract_region(learn_dataset_retired_jobs, "JOB_DEP", "REG_JOB")
learn_dataset_retired_jobs = merge_and_extract_region(learn_dataset_retired_jobs, "FORMER_DEP", "REG_FORMER")

test_dataset_job = merge_and_extract_region(test_dataset_job, "JOB_DEP", "REG_JOB")
test_dataset_retired_jobs = merge_and_extract_region(test_dataset_retired_jobs, "JOB_DEP", "REG_JOB")
test_dataset_retired_jobs = merge_and_extract_region(test_dataset_retired_jobs, "FORMER_DEP", "REG_FORMER")

In [400]:
#Economic sector into fewer categories (and numeric instead of object/string)
def sector_mapping(nace_code):
    if nace_code == "AZ":  
        return "Agriculture, forestry and fishing)"
    elif "BZ" <= nace_code <= "EZ":
        return "Manufacturing, mining and quarrying and other industrial activities"
    elif nace_code == "FZ": 
        return "Construction"
    elif "GZ" <= nace_code <= "IZ":  
        return "Wholesale and retail trade, transportation and storage, accommodation and food service activities"
    elif "JA" <= nace_code <= "JC":
        return "Information and communication"
    elif nace_code == "KZ": 
        return "Financial and insurance activities"
    elif nace_code == "LZ": 
        return "Real estate activities"
    elif "MA" <= nace_code <= "NZ":
        return "Professional, scientific, technical, administrative and support service activities"
    elif "OZ" <= nace_code <= "QB":
        return "Public administration and defence, education, human health and social work activities"
    elif "RZ" <= nace_code <= "UZ":
        return "Other services activities"
    else:
        return "Unknown Sector"

In [401]:
code_Economic_sector["Nomenclature"] = code_Economic_sector["Code"].map(sector_mapping)
code_Economic_sector["Economic_sector_num"] = pd.factorize(code_Economic_sector["Nomenclature"])[0] + 1

code_HIGHEST_CREDENTIAL["HIGHEST_CREDENTIAL_num"] = pd.factorize(code_HIGHEST_CREDENTIAL["Code"])[0] + 1
code_act["act_num"] = pd.factorize(code_act["Code"])[0] + 1

## Merging

In [402]:
learn_data = learn_dataset

learn_data = pd.merge(learn_data, code_act, left_on="act", right_on="Code", how="left")
learn_data.drop(["Code", "Libellé"], axis=1, inplace=True)
learn_data = pd.merge(learn_data, code_HIGHEST_CREDENTIAL, left_on="HIGHEST_CREDENTIAL", right_on="Code", how="left")
learn_data.drop(["Code", "Libellé", "HIGHEST_CREDENTIAL"], axis=1, inplace=True)

#for imputation fitting
learn_data = pd.merge(learn_data, city_pop, on="INSEE_CODE", how="left")
learn_data = pd.merge(learn_data, city_loc, on="INSEE_CODE", how="left")


test_data = test_dataset

test_data = pd.merge(test_data, code_act, left_on="act", right_on="Code", how="left")
test_data.drop(["Code", "Libellé"], axis=1, inplace=True)
test_data = pd.merge(test_data, code_HIGHEST_CREDENTIAL, left_on="HIGHEST_CREDENTIAL", right_on="Code", how="left")
test_data.drop(["Code", "Libellé", "HIGHEST_CREDENTIAL"], axis=1, inplace=True)

In [403]:
learn_dfs = [learn_dataset_emp_contract, learn_dataset_job, learn_dataset_retired_former, learn_dataset_retired_jobs, learn_dataset_retired_pension, learn_sports]
test_dfs = [test_dataset_emp_contract, test_dataset_job, test_dataset_retired_former, test_dataset_retired_jobs, test_dataset_retired_pension, test_sports]

for df in learn_dfs:
    learn_data = pd.merge(learn_data, df, on="PRIMARY_KEY", how="outer")

for df in test_dfs:
    test_data = pd.merge(test_data, df, on="PRIMARY_KEY", how="outer")

In [404]:
# Function to combine columns ending with `_x` and `_y` into a single base column
def combine_duplicate_columns(dataframe):
    for column in dataframe.columns:
        if column.endswith('_x'):
            base_column = column[:-2]  # Remove `_x` suffix
            y_column = base_column + '_y'
            if y_column in dataframe.columns:
                # Combine the `_x` and `_y` columns
                dataframe[base_column] = dataframe[column].fillna(dataframe[y_column])
                # Drop the original `_x` and `_y` columns
                dataframe.drop(columns=[column, y_column], inplace=True)
    return dataframe

# Apply the function to both datasets
learn_data = combine_duplicate_columns(learn_data)
test_data = combine_duplicate_columns(test_data)

## Formatting - for linear regression

In [405]:
def household_num(value):
    parts = value.split('|')  # Split the value by '|'
    if parts[1] in {'1', '2', '3'}:  # For M|1|-- to M|3|--
        return int(parts[1])
    elif parts[1] == '4':  # For M|4|1 to M|4|4
        return 4 + (int(parts[2]) - 1)  # 4 + (1-1), 4 + (2-1), etc.
    return None  # Handle unexpected cases gracefully

In [406]:
code_HOUSEHOLD_TYPE['HOUSEHOLD_TYPE_num'] = code_HOUSEHOLD_TYPE['Code'].apply(household_num)
learn_data['HOUSEHOLD_TYPE'] = learn_data['HOUSEHOLD_TYPE'].apply(household_num)
test_data['HOUSEHOLD_TYPE'] = test_data['HOUSEHOLD_TYPE'].apply(household_num)

In [407]:
def combine_columns(primary_col, fallback_col):
    """Combine two columns, filling missing values in the primary column with values from the fallback column."""
    return primary_col.fillna(fallback_col) if fallback_col is not None else primary_col

def preprocess_employee_data(data, economic_sector_code, work_description_map):
    # Extract numeric values from specific string columns
    data["employee_count"] = data["employee_count"].str.extract(r'tr_(\d)')[0].astype("Int64")
    data["Employer_category"] = data["Employer_category"].str.extract(r'ct_(\d)')[0].astype("Int64")
    
    # Merge with economic sector codes
    data = data.merge(economic_sector_code, left_on="Economic_sector", right_on="Code", how="left")
    
    # Merge with work description map and clean up columns
    data = data.merge(work_description_map, left_on="work_description", right_on="N3", how="left")
    data.drop(["work_description", "N3", "N2"], axis=1, inplace=True)
    data["work_description"] = data["N1"].str.extract(r'csp_(\d)')[0].astype("Int64")
    data.drop("N1", axis=1, inplace=True)
    
    # Combine relevant columns for contracts and pay
    data["emp_contract"] = combine_columns(data["emp_contract"], data["former_emp_contract"])
    data["Pay"] = combine_columns(data["Pay"], data["RETIREMENT_PAY"])
    
    return data

# Apply preprocessing to both learn and test datasets
learn_data = preprocess_employee_data(learn_data, code_Economic_sector, code_work_description_map)
test_data = preprocess_employee_data(test_data, code_Economic_sector, code_work_description_map)

In [408]:
learn_data["sex"] = pd.factorize(learn_data["sex"])[0]
learn_data["studying"] = learn_data["studying"].astype("int64")
learn_data["Sports_Category"] = learn_data["Sports_Category"].fillna(0).astype("int64")
learn_data["REG_JOB"] = pd.to_numeric(learn_data["REG_JOB"], errors='coerce').astype('Int64')
learn_data["REG_FORMER"] = pd.to_numeric(learn_data["REG_FORMER"], errors='coerce').astype('Int64')
learn_data["retirement_age"] = pd.to_numeric(learn_data["retirement_age"], errors='coerce').astype('Int64')
learn_data["WORKING_HOURS"] = pd.to_numeric(learn_data["WORKING_HOURS"], errors='coerce').astype('Int64')
learn_data["Economic_sector_num"] = pd.to_numeric(learn_data["Economic_sector_num"], errors='coerce').astype('Int64')
learn_data["Pay"] = pd.to_numeric(learn_data["Pay"], errors='coerce').astype('Int64')

In [409]:
def replace_na_with_category(column_name):
    global learn_data  # Ensures we modify the global learn_data directly

    # Convert the column to categorical
    learn_data[column_name] = learn_data[column_name].astype('category')
    
    # Define categories to add
    additional_categories = ['Unemployed', 'Retired_Missing', 'Employed_Missing']
    
    # Add the specified categories
    learn_data[column_name] = learn_data[column_name].cat.add_categories(additional_categories)
    
    learn_data.loc[(learn_data[column_name].isna()) & (learn_data['JOB_42'].astype(str).str.startswith('csp_7')), column_name] = 'Retired_Missing'
    learn_data.loc[(learn_data[column_name].isna()) & (learn_data['act_num'] == 1), column_name] = 'Employed_Missing'
    learn_data.loc[(learn_data[column_name].isna()) & ((learn_data['JOB_42'].astype(str).str.startswith('csp_8')) | (learn_data['act_num'] == 2)), column_name] = 'Unemployed'

In [410]:
replace_na_with_category("emp_contract")
replace_na_with_category("TYPE_OF_CONTRACT")
replace_na_with_category("WORK_CONDITION")
replace_na_with_category("labor_force_status")
replace_na_with_category("Economic_sector_num")
replace_na_with_category("REG_JOB")
replace_na_with_category("REG_FORMER")
replace_na_with_category("work_description")

In [411]:
learn_data['Employer_category'] = learn_data.apply(
    lambda row: 10 if pd.isna(row['Employer_category']) and 
                        (str(row['JOB_42']).startswith('csp_8') or row['act_num'] == 2)  # unemployed
                else (0 if pd.isna(row['Employer_category']) else row['Employer_category']),  # employed and retired missing
    axis=1
)

learn_data['employee_count'] = learn_data.apply(
    lambda row: 7 if pd.isna(row['employee_count']) and 
                        (str(row['JOB_42']).startswith('csp_8') or row['act_num'] == 2)  # unemployed
                else (0 if pd.isna(row['employee_count']) else row['employee_count']),
    axis=1
)


In [412]:
learn_data.loc[learn_data['JOB_42'].astype(str).str.startswith('csp_7'), 'JOB_42'] = learn_data['FORMER_JOB_42']
learn_data.loc[(learn_data['emp_contract'] == 'Unemployed') & (learn_data['Pay'].isna()), 'Pay'] = 0
learn_data.loc[(learn_data['emp_contract'] == 'Unemployed') & (learn_data['WORKING_HOURS'].isna()), 'WORKING_HOURS'] = 0

learn_data = learn_data.drop(columns=["act", "former_emp_contract", "RETIREMENT_PAY", "retirement_age", "Economic_sector", "Code", "Libellé", "Nomenclature", "INSEE_CODE", "X", "Y"])

In [413]:
# Define job-specific pay and working hours
job_defaults = {
    'csp_1': {'Pay': 50000, 'WORKING_HOURS': 2860},  # Agriculture
    'csp_2_1': {'Pay': 24000, 'WORKING_HOURS': 2288},  # Artisans
    'csp_2_2': {'Pay': 39937, 'WORKING_HOURS': 2444},  # Commerçant
    'csp_2_3': {'Pay': 58248, 'WORKING_HOURS': 2704},  # Chefs d'enterprise
}

In [414]:

# Function to fill missing values for Pay and WORKING_HOURS
def fill_job_defaults(data, job_defaults):
    for job, defaults in job_defaults.items():
        # Handle string startswith for specific cases (e.g., 'csp_1')
        job_condition = data['JOB_42'].astype(str).str.startswith(job) if '_' not in job else (data['JOB_42'] == job)
        
        # Fill missing Pay and WORKING_HOURS
        data.loc[job_condition & data['Pay'].isna(), 'Pay'] = defaults['Pay']
        data.loc[job_condition & data['WORKING_HOURS'].isna(), 'WORKING_HOURS'] = defaults['WORKING_HOURS']

# Apply the function to both datasets
fill_job_defaults(learn_data, job_defaults)
fill_job_defaults(test_data, job_defaults)

## Encoding

In [415]:
#for linear regression I need only number 

#do one-hot encoding for "labor_force status", but with catagegory being 0, Employed_missing, Unemployed and Other (=only being 6%)
#do one hot-encodung for "type_of_contratct" with CDI, Unemployed, Employed_Missing, Other (=only being 12%)
#do one_hot_encding for "work_condition" with Unemployed, P, Other(=only being 17%)
#do one hot_econding for "emp_contract", with EMP1-6, Unemployed, EMP2-1, Other (=being 12%)
#can keep as is for "highest_credential" because the ordinality make sense here for education
#same for the location of insee code 
#use "work condition" instead of JOB_42 and then do one-hot-encdoding
#do one-hot-encoding for "household_type"
#for "act" do one hot encoding but maybe merge stay at home people with inactif + drop less than 14years olds as nobody in our data is 
#do one hot encoding for "sport" but with a "Other" category
#do one hot encdoding for "REG_JOB" and "FORMER_REG"
#do one hot encdoding for "employer_category" with 1 to 7 being "Others
#can keep it as is for "employee count" because here the ordinality make sense
#do one hot encoding for "Economic_sector_num" and "work_description"


In [416]:
# Custom Transformer for Handling Imputation with Averages
class ImputeMissingValues(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Calculate averages for training data (ignoring zero values)
        self.avg_working_hours = X.loc[X["WORKING_HOURS"] != 0].groupby("JOB_42")["WORKING_HOURS"].mean().round()
        self.avg_pay = X.loc[X["Pay"] != 0].groupby("JOB_42")["Pay"].mean().round()
        return self

    def transform(self, X, y=None):
        # Use the stored averages to impute missing values
        X = X.copy()
        X["WORKING_HOURS"] = X.apply(
            lambda row: self.avg_working_hours.get(row["JOB_42"], np.nan) 
            if pd.isnull(row["WORKING_HOURS"]) else row["WORKING_HOURS"],
            axis=1
        )
        X["Pay"] = X.apply(
            lambda row: self.avg_pay.get(row["JOB_42"], np.nan) 
            if pd.isnull(row["Pay"]) else row["Pay"],
            axis=1
        )
        return X

In [417]:
# Custom Transformer for MICE (Multiple Imputation by Chained Equations) Imputation
class MICEImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_impute):
        self.columns_to_impute = columns_to_impute
        self.imputer = IterativeImputer(random_state=100)
        
    def fit(self, X, y=None):
        # Fit the imputer only on the specified columns
        self.imputer.fit(X[self.columns_to_impute])
        return self
    
    def transform(self, X):
        # Transform only the specified columns
        X_imputed = X.copy()
        X_imputed[self.columns_to_impute] = self.imputer.transform(X[self.columns_to_impute])
        return X_imputed

In [418]:
# Specify the columns that need MICE imputation
columns_to_impute = ["Employer_category", "employee_count"]

# Define the combined pipeline
pipeline = Pipeline([
    ('impute_averages', ImputeMissingValues()),  # Step 1: Impute using averages
    ('mice_imputer', MICEImputer(columns_to_impute=columns_to_impute)),  # Step 2: MICE imputation
])

pipeline.fit_transform(learn_data)

,PRIMARY_KEY,sex,JOB_42,studying,age_2020,HOUSEHOLD_TYPE,target,act_num,HIGHEST_CREDENTIAL_num,RESIDENTS,...,Sports_Category,Employer_category,employee_count,TYPE_OF_CONTRACT,WORK_CONDITION,labor_force_status,WORKING_HOURS,REG_JOB,Economic_sector_num,work_description
0,1,0,csp_5_4,0,34,3,9.367020,1,10,14514,...,1,9.0,1.0,CDI,C,O,1470.0,84,7,5
1,5,1,csp_6_3,0,80,7,8.648771,3,6,14514,...,0,9.0,1.0,CDI,C,O,793.0,44,4,6
2,7,0,csp_3_1,0,63,4,10.792503,1,11,14514,...,0,0.0,0.0,Employed_Missing,Employed_Missing,Employed_Missing,1241.0,Employed_Missing,Employed_Missing,Employed_Missing
3,8,1,csp_3_7,0,50,4,8.508222,1,9,14514,...,0,9.0,1.0,CDI,C,O,1201.0,11,2,3
4,10,0,csp_5_6,0,68,7,8.966272,2,6,14514,...,0,10.0,7.0,Unemployed,Unemployed,Unemployed,0.0,Unemployed,Unemployed,Unemployed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50038,100077,0,csp_8_5,1,29,5,11.154438,6,6,28540,...,0,10.0,7.0,Unemployed,Unemployed,Unemployed,0.0,Unemployed,Unemployed,Unemployed
50039,100078,0,csp_4_3,0,34,4,9.962345,1,9,28540,...,0,8.0,6.0,CDI,P,O,665.0,11,9,4
50040,100079,0,csp_5_2,0,29,6,12.226288,1,8,28540,...,0,8.0,4.0,CDI,C,O,3000.0,11,9,5
50041,100081,1,csp_8_5,0,57,3,8.965529,7,6,28540,...,0,10.0,7.0,Unemployed,Unemployed,Unemployed,0.0,Unemployed,Unemployed,Unemployed


In [419]:
#this gives 0 and 1

def preprocess_for_regression(data):
    # Initialize OneHotEncoder with sparse output disabled to return dense arrays
    ohe = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
    
    # List of columns to encode
    categorical_columns = [
        'HOUSEHOLD_TYPE', 'act_num', 'Sports_Category', 
        'Employer_category', "employee_count"
    ]

    # List of columns to encode categorical_columns = [
        #'labor_force_status', 'TYPE_OF_CONTRACT', 'WORK_CONDITION', 
        #'emp_contract', 'HOUSEHOLD_TYPE', 'act_num', 'Sports_Category', 
        #'REG_JOB', 'REG_FORMER', 'Employer_category', 'Economic_sector_num', 
        #'work_description' ]
    
    # Apply the encoder
    encoded_data = ohe.fit_transform(data[categorical_columns])
    
    # Create a DataFrame for the encoded features
    encoded_feature_names = ohe.get_feature_names_out(categorical_columns)
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names, index=data.index)
    
    # Drop original categorical columns from the dataset
    data = data.drop(columns=categorical_columns)
    
    # Merge encoded features with the original dataset
    processed_data = pd.concat([data, encoded_df], axis=1)
    
    return processed_data


# Apply preprocessing to learn and test datasets
learn_data = preprocess_for_regression(learn_data)
#test_data = preprocess_for_regression(test_data)


In [420]:
#we might need to rescale working_hours, pay and employee_count
#risk multicollinearity so check for highly correlated features and consider dropping or combining them.
#might need to use PCA to reduce dimensionality

## Prediction

In [421]:
x_train = learn_data.drop(columns=["target"])
y_train = learn_data["target"]

In [422]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train,
                                                    y_train, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [423]:
cv_folds = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize the RandomForestRegressor model
model = LinearRegression()  #need to add that in the pipeline 

# Set up the GridSearchCV for RandomForestRegressor with appropriate scoring metric
#rf_search = GridSearchCV(model, cv=cv_folds, n_jobs=-1, scoring="mean_squared_error")

In [424]:
rf_res = model.fit(X_train, Y_train)

TypeError: boolean value of NA is ambiguous

In [ ]:
# Get the best model after GridSearchCV
#best_model_rf = rf_res.best_estimator_

# Output the best model and its score
print(f"Best model: {rf_res}")
print(f"Best score: {rf_res.best_score_}")

In [ ]:
train_predictions = rf_res.predict(X_train)
print("R² for training set: ", r2_score(Y_train, train_predictions))
print("RMSE on the learning set:", root_mean_squared_error(Y_train, rf_res.predict(X_train)))